<a href="https://colab.research.google.com/github/harshitxjangam/Imageclass_DL/blob/main/training_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# create folder structure
!mkdir -p data models figures notebooks
!echo "Project folders created successfully ✅" && ls -l


Project folders created successfully ✅
total 20
drwxr-xr-x 2 root root 4096 Oct 15 03:29 data
drwxr-xr-x 2 root root 4096 Oct 15 03:29 figures
drwxr-xr-x 2 root root 4096 Oct 15 03:29 models
drwxr-xr-x 2 root root 4096 Oct 15 03:29 notebooks
drwxr-xr-x 1 root root 4096 Oct 13 13:43 sample_data


In [2]:
%%writefile README.md
# 🎶 Musical Instruments Image Classification

This project trains multiple deep learning models (ResNet18, MobileNetV2, ViT)
on a musical instruments dataset and explores inference optimization techniques
(TorchScript, ONNX, and INT8 quantization).


Writing README.md
